# Homework Starter — Stage 05: Data Storage
Name: Yuqing Yan
Date: 08/17/2025

Objectives:
- Env-driven paths to `data/raw/` and `data/processed/`
- Save CSV and Parquet; reload and validate
- Abstract IO with utility functions; document choices

In [1]:
import os, pathlib, datetime as dt
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# 1. Initialize environment paths
load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

RAW -> /Users/yuqingyan/Desktop/bootcamp_Yuqing_Yan/homework/homework5/data/raw
PROC -> /Users/yuqingyan/Desktop/bootcamp_Yuqing_Yan/homework/homework5/data/processed


## 1) Create or Load a Sample DataFrame
You may reuse data from prior stages or create a small synthetic dataset.

In [4]:
# 2. Create sample DataFrame
def create_sample_data():
    np.random.seed(42)
    dates = pd.date_range('2024-01-01', periods=10, freq='D')
    return pd.DataFrame({
        'date': dates,
        'ticker': ['AAPL']*5 + ['MSFT']*5,
        'price': np.round(150 + np.random.randn(10).cumsum(), 2),
        'volume': np.random.randint(10000, 50000, 10)
    })

df = create_sample_data()
print("\nSample DataFrame:")
print(df.head())


Sample DataFrame:
        date ticker   price  volume
0 2024-01-01   AAPL  150.50   15311
1 2024-01-02   AAPL  150.36   47819
2 2024-01-03   AAPL  151.01   49188
3 2024-01-04   AAPL  152.53   27568
4 2024-01-05   AAPL  152.30   29769


## 2) Save CSV to data/raw/ and Parquet to data/processed
- Use timestamped filenames.
- Handle missing Parquet engine gracefully.

In [5]:
# 3. Save in both formats
def get_timestamp():
    return dt.datetime.now().strftime('%Y%m%d_%H%M%S')

# Save CSV to raw
csv_filename = f"stock_data_{get_timestamp()}.csv"
csv_path = RAW / csv_filename
df.to_csv(csv_path, index=False)
print(f"\nSaved CSV to: {csv_path}")

# Save Parquet to processed
pq_filename = f"stock_data_{get_timestamp()}.parquet"
pq_path = PROC / pq_filename
try:
    df.to_parquet(pq_path, engine='pyarrow')
    print(f"Saved Parquet to: {pq_path}")
except Exception as e:
    print(f"\nParquet save failed (install pyarrow/fastparquet): {e}")
    pq_path = None
pq_path


Saved CSV to: data/raw/stock_data_20250817_135352.csv
Saved Parquet to: data/processed/stock_data_20250817_135352.parquet


PosixPath('data/processed/stock_data_20250817_135352.parquet')

## 3) Reload and Validate (TODO)
- Compare shapes and key dtypes.

In [ ]:
# 4. Reload and validate
def validate_reloaded(original, reloaded):
    """Validate shape, dtypes and critical columns"""
    validation = {
        'shapes_match': original.shape == reloaded.shape,
        'columns_match': set(original.columns) == set(reloaded.columns),
        'date_dtype_ok': pd.api.types.is_datetime64_any_dtype(reloaded['date']),
        'price_dtype_ok': pd.api.types.is_float_dtype(reloaded['price']),
        'volume_dtype_ok': pd.api.types.is_integer_dtype(reloaded['volume']),
        'data_integrity': original.equals(reloaded)
    }
    return pd.Series(validation)

# Validate CSV
df_csv = pd.read_csv(csv_path, parse_dates=['date'])
print("\nCSV Validation Results:")
print(validate_reloaded(df, df_csv))

In [ ]:
if pq_path:
    try:
        df_pq = pd.read_parquet(pq_path)
        validate_loaded(df, df_pq)
    except Exception as e:
        print('Parquet read failed:', e)

## 4) Utilities (TODO)
- Implement `detect_format`, `write_df`, `read_df`.
- Use suffix to route; create parent dirs if needed; friendly errors for Parquet.

In [ ]:
import typing as t, pathlib

def detect_format(path: t.Union[str, pathlib.Path]):
    s = str(path).lower()
    if s.endswith('.csv'): return 'csv'
    if s.endswith('.parquet') or s.endswith('.pq') or s.endswith('.parq'): return 'parquet'
    raise ValueError('Unsupported format: ' + s)

def write_df(df: pd.DataFrame, path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    fmt = detect_format(p)
    if fmt == 'csv':
        df.to_csv(p, index=False)
    else:
        try:
            df.to_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e
    return p

def read_df(path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path)
    fmt = detect_format(p)
    if fmt == 'csv':
        return pd.read_csv(p, parse_dates=['date']) if 'date' in pd.read_csv(p, nrows=0).columns else pd.read_csv(p)
    else:
        try:
            return pd.read_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e

# Demo
p_csv = RAW / f"util_{ts()}.csv"
p_pq  = PROC / f"util_{ts()}.parquet"
write_df(df, p_csv); read_df(p_csv).head()
try:
    write_df(df, p_pq)
    read_df(p_pq).head()
except RuntimeError as e:
    print('Skipping Parquet util demo:', e)

## 5) Documentation (TODO)
- Update README with a **Data Storage** section (folders, formats, env usage).
- Summarize validation checks and any assumptions.